In [17]:
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.util import Inches
import pandas as pd
from io import BytesIO

In [18]:
# def apply_gradient_transparency(input_path, mask="left_to_right"):
#     img = Image.open(input_path).convert("RGBA")
#     width, height = img.size
    
#     gradient = Image.new("L", (width, height), 0)
#     draw = ImageDraw.Draw(gradient)
#     pixels = gradient.load()
    
    
#     if mask=="left_to_right":
#         for x in range(width):
#             alpha = int(255 * (1 - x / width))
#             draw.line([(x, 0), (x, height)], fill=alpha)
#     elif mask=="right_to_left":
#         for x in range(width,0,-1):
#             alpha = int(255 * (x / width))
#             draw.line([(x, 0), (x, height)], fill=alpha)
#     if mask=="top_left_to_bottom_right":
#         for x in range(width):
#             for y in range(height):
#                 ratio_x = x / (width - 1) if width > 1 else 0
#                 ratio_y = y / (height - 1) if height > 1 else 0
#                 alpha = int(255 * (ratio_x + ratio_y) / 2)
#                 pixels[x, y] = alpha
#     if mask=="top_right_to_bottom_left":
#         for x in range(width):
#             for y in range(height):
#                 ratio_x = x / (width - 1) if width > 1 else 0
#                 ratio_y = y / (height - 1) if height > 1 else 0
#                 alpha = int(255 * (1 - ratio_x + 1 - ratio_y) / 2)
#                 pixels[x, y] = alpha
#     # 应用渐变透明效果
#     img.putalpha(gradient)
#     return img

In [161]:
def concatenae(canvas,image_a,image_b,total_width,max_height):
    canvas.paste(image_a, (0, 0))
    canvas.paste(image_b, (image_a.width, 0))
    gradient = Image.new("L", (total_width, max_height), 0)
    draw = ImageDraw.Draw(gradient)
    for x in range(image_a.width):
        alpha = max(0,int(255 * (1 - x / image_a.width)))
        draw.line([(x, 0), (x, max_height)], fill=alpha)
    for x in range(image_b.width):
        alpha = max(0,int(255 * (x / image_b.width)))
        draw.line([(image_a.width+x, 0), (image_a.width+x, max_height)], fill=alpha)
    canvas.putalpha(gradient)
    return canvas

def add_subtitle(img,text,font="arial.ttf",font_size=25,height_percent=0.8):
    draw = ImageDraw.Draw(img)
    width, height = img.size
    # 样式
    font = ImageFont.truetype(font, font_size)
    text_color = (255, 255, 255)
    border_color = "#4874CB"
    border_width = 1

    text_width = max([draw.textlength(i, font=font) for i in text.split("\n")])
    x = (width - text_width) / 2
    y = height*height_percent

    # 边框
    for dx in range(-border_width, border_width+1):
        for dy in range(-border_width, border_width+1):
            if dx != 0 or dy != 0:
                draw.text((x+dx, y+dy), text, font=font, fill=border_color)
    
    # 文字
    draw.text((x, y), text, font=font, fill=text_color)
    return img

def generate_background_image(image_a, image_b,image_a_dark=False,image_b_dark=False):
    max_width = max(image_a.width, image_b.width)
    image_a = image_a.resize((max_width, int(image_a.height * (max_width / image_a.width))))
    image_b = image_b.resize((max_width, int(image_b.height * (max_width / image_b.width))))
    enhancer_a = ImageEnhance.Brightness(image_a)
    enhancer_b = ImageEnhance.Brightness(image_b)
    if image_a_dark:
        image_a = enhancer_a.enhance(0.3)
    if image_b_dark:
        image_b = enhancer_b.enhance(0.3)
    
    total_width = image_a.width + image_b.width
    max_height = max(image_a.height, image_b.height)
    
    new_image = Image.new('RGB', (total_width, max_height))
    
    new_image = concatenae(new_image,image_a,image_b,total_width,max_height)
    return new_image

In [162]:
df = pd.read_excel("Dialogue 翻译后.xlsx")

In [163]:
l = Image.open("left.png").convert("RGBA")
r = Image.open("right.png").convert("RGBA")
# new_image = generate_background_image(l,r)

In [56]:
df.head()

,RIGHT,LEFT,RIGHT_English,LEFT_English,duration
0,Hello,NaN,Hello,NaN,2
1,NaN,...你一直在跟踪他。,NaN,...You've been tracking him.,2
2,是的，从1964年起，我便在苏联境内清理他留下的痕迹。,NaN,"Yes, since 1964 in the Soviet Union, I've bee...",NaN,8.5
3,Snake总能完成任务，而我则确保一切不留证据。,NaN,"Snake always complete his task , and I make su...",NaN,6.5
4,NaN,清理痕迹...倒是个称职的清道夫。,NaN,Cleaning up the trail... Rather a competent sc...,6


In [62]:
prs = Presentation("Sample_do_not_delete.pptx")
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)
left = top = Inches(0)

byte_stream = BytesIO()
new_image = generate_background_image(l,r)
new_image.save(byte_stream, format='PNG')

pic = slide.shapes.add_picture(byte_stream, left, top, width=prs.slide_width, height=prs.slide_height)

slide.shapes._spTree.remove(pic._element)
slide.shapes._spTree.insert(2, pic._element)

In [63]:
for i in df.index:
    l = Image.open("left.png").convert("RGBA")
    r = Image.open("right.png").convert("RGBA")
    if pd.isna(df.loc[i,"LEFT"]):
        current_subtitle = df.loc[i,"RIGHT"]
        current_subtitle_english = df.loc[i,"RIGHT_English"]
        new_img = generate_background_image(l,r,image_a_dark=True)
    else:
        current_subtitle = df.loc[i,"LEFT"]
        current_subtitle_english = df.loc[i,"LEFT_English"]
        new_img = generate_background_image(l,r,image_b_dark=True)

    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    left = top = Inches(0)
    new_img = add_subtitle(new_img,current_subtitle,font_size=50,height_percent=0.8,font="simfang.ttf")
    new_img = add_subtitle(new_img,current_subtitle_english,font_size=40,height_percent=0.9)
    byte_stream = BytesIO()
    new_img.save(byte_stream, format='PNG')
    pic = slide.shapes.add_picture(byte_stream, left, top, width=prs.slide_width, height=prs.slide_height)
    slide.shapes._spTree.remove(pic._element)
    slide.shapes._spTree.insert(2, pic._element)

In [64]:
prs.save("result_1.pptx")

In [86]:
from pptx.oxml import parse_xml
import re

In [155]:
# 保存为xml文件，后续修改xml以设置自动放映时长
with open("./result_1.xml") as f:
    data = f.read()

In [156]:
data = data.split("</p:clrMapOvr></p:sld>")

In [157]:
merge_cells = ['</p:clrMapOvr></p:sld>']+[f'</p:clrMapOvr><p:transition advTm="{int(df.loc[i,"duration"]*1000)}"/></p:sld>' for i in range(df.shape[0])]

In [ ]:
res = data[0]
for i in range(len(merge_cells)):
    idx = int(re.findall('pkg:name="/ppt/slides/_rels/slide(.*?).xml.rels"',data[i])[0]) # xml中的顺序不是播放顺序，所以需要根据正则表达式来调整顺序
    x = merge_cells[idx-1]
    res += x+data[i+1]

In [160]:
# 保存为xml文件
with open("./result_2.xml","w") as f:
    f.write(res)

In [129]:
merge_cells

['</p:clrMapOvr></p:sld>',
 '</p:clrMapOvr><p:transition advTm="2000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="2000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="8500"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="6500"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="6000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="3000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="6000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="5000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="4000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="5000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="7000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="4000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="5000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="6000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="3200"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="6000"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="4800"/></p:sld>',
 '</p:clrMapOvr><p:transition advTm="4

In [77]:
len('</p:clrMapOvr></p:sld>')

22

In [47]:
from mutagen.mp3 import MP3
import os

In [50]:
res = 0
for i in range(15):
    audio = MP3(f"D:/git_projects/RolePlayBetweenAI_py/Skull Face/Altered/{i}.mp3")
    print(f"Skull Face_{i}:{audio.info.length}")
    res += audio.info.length
    audio = MP3(f"D:/git_projects/RolePlayBetweenAI_py/The Boss/Altered/{i}.mp3")
    print(f"The Boss_{i}:{audio.info.length}")
    res += audio.info.length

Skull Face_0:1.5934375
The Boss_0:1.8546875
Skull Face_1:14.696145124716553
The Boss_1:5.783310657596372
Skull Face_2:13.221133786848073
The Boss_2:3.739183673469388
Skull Face_3:20.75578231292517
The Boss_3:5.184897959183673
Skull Face_4:13.831836734693878
The Boss_4:3.915827664399093
Skull Face_5:11.349251700680272
The Boss_5:4.940816326530612
Skull Face_6:7.684308390022676
The Boss_6:5.445034013605442
Skull Face_7:6.7798185941043085
The Boss_7:7.680453514739229
Skull Face_8:15.361315192743763
The Boss_8:11.297460317460317
Skull Face_9:15.46249433106576
The Boss_9:2.50775
Skull Face_10:31.147845804988663
The Boss_10:33.3087074829932
Skull Face_11:133.25990929705216
The Boss_11:87.06013605442178
Skull Face_12:158.95845804988662
The Boss_12:101.80331065759637
Skull Face_13:45.74566893424036
The Boss_13:20.012562358276643
Skull Face_14:5.937551020408163
The Boss_14:6.446848072562358


In [ ]:
print(res)